In [1]:
!pip install openai --quiet
!pip install mosestokenizer --quiet
!pip install nltk --quiet

In [2]:
import config
import json

In [3]:
import openai

openai.api_key = config.api_key

In [4]:
# 'Substitute offensive/hate terms in the tweet with inclusive language and generate JSON of a few rephrased versions of the tweet, and list down the suspected problematic terms:'

In [5]:
# 04 Dec 2023 and 05 Dec 2023, UTC based

In [6]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
import mosestokenizer
import numpy as np

In [7]:
df = pd.read_csv(config.VIDH_DATAFILE, index_col=0)

In [8]:
df.shape

(13104, 2)

In [9]:
df.head()

,text,Label
id,,
acl11159,There is way too many m0ng0l0id kids at my sch...,1
acl11161,the worst of them are the taaaaaaaaaards becau...,1
acl11163,it is not niiiice to be around reeetaaaardss,1
acl11165,travellers have a certain aroma,1
acl11167,Can you smell these dirty muzzies? It's a brut...,1


In [10]:
df['Label'].value_counts()

Label
1    6624
0    6480
Name: count, dtype: int64

In [17]:
tk = TweetTokenizer()
detk = mosestokenizer.MosesDetokenizer('en')
t = df.iloc[3390].to_list()[0]
tk = tk.tokenize(t)
display(t)
detk(tk)

'you are breastfeeding you should go to the changing room to feed your baby'

'you are breastfeeding you should go to the changing room to feed your baby'

In [18]:
def get_offensive_words():
    _df = pd.read_csv(config.en_swear_words_datafile, index_col=0)
    
    s = np.logical_or(_df['Level of offensiveness']=='Strongest words', _df['Level of offensiveness']=='Strong words')
    # display(_df[s]['Word'].to_list())
    wd_list = _df['Word'].to_list()
    
    _df = pd.read_csv(config.en_profanity_datafile, index_col=None)
    s = _df['severity_description'] == 'Severe'
    # wd_list.extend(_df[s]['text'].to_list())
    wd_list.extend(_df['text'].to_list())
    wd_list = set(map(str.lower, wd_list))
    return wd_list
    
offensive_wd_list = get_offensive_words()

In [19]:
def find_phrases(tokens, phrases):
    tokens = list(map(str.lower, tokens))
    """
    Find phrases in a list of sequential tokens.
    
    Args:
        tokens (list): List of sequential tokens.
        phrases (list): List of phrases to search for.
        
    Returns:
        A list of tuples containing the start and end index of each found phrase.
    """
    found_phrases = []
    
    for i in range(len(tokens)):
        for phrase in phrases:
            if tokens[i:i+len(phrase)] == phrase:
                found_phrases.append((i, i+len(phrase)-1))
    
    return found_phrases

def replace_phrases(tk, found_phrases):
    # print(found_phrases)
    tk_out = tk[:]
    off_phrases = []
    for m in found_phrases:
        tk_out[m[1]] = tk[m[1]] + tk[m[1]][-1]
        off_phrases.append(tk[m[0]:m[1]+1])
    return tk_out, off_phrases


def preprocess(t):
    tk = TweetTokenizer()
    detk = mosestokenizer.MosesDetokenizer('en')
    tk = tk.tokenize(t)
    # print(tk)
    phrase_index = find_phrases(tk, list(map(str.split, offensive_wd_list)))
    tk_output = replace_phrases(tk, phrase_index)
    # display(t)
    # fix \n error in tokenizer patch started
    tk_output = list(tk_output)
    tk_output[0] = list(map(lambda x:x.replace('\n', ''), tk_output[0]))
    tk_output = tuple(tk_output)
    # fix \n error in tokenizer patch finished
    return detk(tk_output[0]), phrase_index, tk_output[1]

t, m, o = preprocess("@goll what a Mother fucker, and ass-fuckers")
t, m, o

('@goll what a Mother fuckerr, and ass-fuckerss',
 [(3, 4), (4, 4), (7, 7)],
 [['Mother', 'fucker'], ['fucker'], ['ass-fuckers']])

In [20]:
print(df['Label'].value_counts())
problematic_df = df[df['Label']>0]
problematic_df.shape

Label
1    6624
0    6480
Name: count, dtype: int64


(6624, 2)

In [21]:
gtp_dict = {}
gpt_response_file = config.gpt_response_vidh_file
out_file = open(gpt_response_file, "r")
gtp_dict = json.load(out_file)
out_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'out/gpt-response-vidh.json'

In [22]:
def chatgpt_prompt(msg):
    openai.api_key = config.api_key
    messages = [ {"role": "system", "content":
                  "You are a intelligent assistant."},
               ]
    message = msg
    messages.append({"role": "user", "content": message})
    messages.append({"role": "assistant", "content": 'You reply in JSON only, no free text.'})
    # print(messages)
    chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=2048, top_p=1, frequency_penalty=0, presence_penalty=0)
    reply = chat.choices[0].message.content
    return reply, chat

In [23]:
import time
unit_delay = 3
error_delay = 60 * 10

In [27]:
base_msg = 'Substitute problematic terms in the following tweets with inclusive language and produce a list of a few rephrased versions of the tweet and list down suspected problematic terms:\n\n'

bunch = 5
tot = problematic_df.shape[0]
# tot = 400
while True:
    try:
        for i in range(0, tot, bunch):
            if str(i) in gtp_dict:
                continue
            print(i)
            time.sleep(unit_delay)
            message = base_msg
            for j in range(i, min(tot, i+bunch)):
                t = problematic_df.iloc[j].to_list()[0]
                # print(t)
                twt, m, o = preprocess(t)
                message += str(j) +'. ' + twt + '\n'
            # print(message)
            reply, chat = chatgpt_prompt(message)
            gtp_dict[str(i)] = reply
            out_file = open(gpt_response_file, "w")
            json.dump(gtp_dict, out_file, indent = 2)
            out_file.close()
        break
    except Exception as error:
        print('error:', error)
        time.sleep(error_delay)
        pass

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
error: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
110